In [ ]:
pip install langchain openai

In [ ]:
pip install langchain-community


In [ ]:
# pinecone 설치
!pip install pinecone

In [ ]:
pip install kiwipiepy

In [ ]:
pip install langchain-upstage

In [ ]:
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from kiwipiepy import Kiwi
from pinecone import Pinecone
from langchain_upstage import UpstageEmbeddings
import json

# Pinecone Index 이름
index_name = "kb-docs"  # 기존에 생성된 Index 이름

# Pinecone 초기화
pc = Pinecone(api_key="")
index = pc.Index(index_name)  # 기존 Index 가져오기


# OpenAI API 키 설정
api_key = ""

# LangChain의 OpenAI LLM 설정
chat = ChatOpenAI(
    model="gpt-4o",  # 또는 "gpt-4o"
    openai_api_key=api_key,
    temperature=0.7,
    max_tokens=1000
)


query_text = input("")
kiwi = Kiwi()
keywords = [token.form for token in kiwi.tokenize(query_text) if token.tag in ['NNG', 'NNP']]  # 일반 명사와 고유 명사 추출

# 기본 file_detail 값 생성
file_detail_values = []

# 조건에 따라 file_detail 값 추가
if "대출" in keywords or "융자" in keywords or "자금" in keywords:
    file_detail_values.extend(["담보대출", "자동차대출", "신용대출"])
if "예금" in keywords or "저축" in keywords or "예치" in keywords:
    file_detail_values.append("예금")
if "적금" in keywords or "정기적금" in keywords or "월저축" in keywords:
    file_detail_values.append("적금")
if "통장" in keywords or "계좌" in keywords or "입출금" in keywords:
    file_detail_values.append("입출금자유")
if "주택" in keywords or "집" in keywords or "부동산" in keywords:
    file_detail_values.append("담보대출")
if "차" in keywords or "자동차" in keywords or "차량" in keywords:
    file_detail_values.append("자동차대출")

# OR 조건으로 필터 생성
metadata_filter = {
    "file_detail": {"$in": file_detail_values}  # file_detail이 리스트 중 하나에 포함
}

# Solar-Embedding 모델 로드
embedding_model = UpstageEmbeddings(
    model="solar-embedding-1-large",
    api_key=""  # API 키를 여기에 입력하세요.
)
# Solar-Embedding으로 쿼리 텍스트를 벡터화
query_vector = embedding_model.embed_query(query_text)

# Pinecone에서 쿼리 실행
namespace = "kb_namespace"  # 검색할 namespace 설정
results = index.query(
    vector=query_vector,     # 쿼리 벡터
    namespace=namespace,     # 검색할 namespace
    top_k=8,                 # 검색 결과 개수
    include_values=False,    # 벡터 값 포함 여부
    include_metadata=True,    # 메타데이터 포함 여부
    filter=metadata_filter  # 메타메이터 필터 on
)

# 데이터 가공 및 병합
grouped_data = {}

for match in results["matches"]:
    file_name = match["metadata"].get("file_name", "")  # 파일 이름
    file_detail = match["metadata"].get("file_detail", "")  # 파일 상세
    file_text = match["metadata"].get("text", "")  # 파일 내용

    # 같은 file_name 그룹화
    if file_name not in grouped_data:
        grouped_data[file_name] = {"file_detail": file_detail, "texts": []}
    grouped_data[file_name]["texts"].append(file_text)

# 데이터 확인
for file_name, data in grouped_data.items():
    print(f"File Name: {file_name}")
    print(f"Category: {data['file_detail']}")
    print("Contents:")
    for text in data["texts"]:
        print(f"- {text[:100]}...")  # 첫 100자만 출력
    print("\n")

# 메시지 정의
system_message = SystemMessage(content=f"""
You are an assistant for question-answering tasks specialized in financial products.
You will be given JSON DATA and user conversation history.

Structure of JSON DATA will be

  Product Name:
  Category:
  Description:

JSON DATA: {grouped_data}
user conversation history :


You must follow the rules below.

1. Each file name represents the product name.
2. The category indicates the type of product.
3. Only use JSON DATA to find relevant product information.
4. Do not mix content between different products.
5. If the product cannot be identified, respond with: "죄송합니다. 해당 상품 정보를 찾을 수 없습니다."
6. If the input is unrelated to financial product consultations, respond with: "저는 KB 국민은행의 금융 상품과 정보만 제공해드릴 수 있습니다. 다시 입력해주세요."
7. If the input concerns financial institutions other than KB 국민은행, respond with: "저는 KB 국민은행의 금융 상품과 정보만 제공해드릴 수 있습니다. 다시 입력해주세요."
8. Answer naturally without including "Product Name", "Category", or "Description" in the response.
9. Base your responses only on the product information and user conversation history.

""")

# 사용자 질문
human_message = HumanMessage(content=query_text)

# LangChain LLM 호출
response = chat([system_message, human_message])

# 응답 출력
print(response.content)